In [73]:
import pandas as pd

import random
import os
import torch.random
import numpy as np

from model import TimeSeriesDetector
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import warnings
from plotting import plot_detection
warnings.filterwarnings('ignore')

In [2]:
def seed_everything(seed=7575):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.random.manual_seed(seed)

seed_everything()

In [3]:
train = pd.read_excel("Задача 1. Датасет 1. Тренировочный неразмеченный.xlsx", index_col='Параметр')
test = pd.read_excel("Задача 1. Датасет 1. Тренировочный размеченный.xlsx", index_col='Параметр')

In [89]:
# detector = TimeSeriesDetector(
#     granularity='PT1H',
#     num_lags=24,
#     model=RandomForestRegressor,
# )


detector = TimeSeriesDetector(
    granularity='PT1H',
    num_lags=24,
    model=Ridge,
    alpha=7,
    sigma=2.3
)

In [90]:
ts_train, ts_test = train.loc[:"2022-01-03 05:20:00", 'х001'], train.loc["2022-01-03 05:20:00":, 'х001']

In [91]:
detector.fit(ts_train)

detector.fit_statistics(ts_train)

In [92]:
preds = detector.predict_batch(ts_train, ts_test)

In [93]:
lower, upper = detector.get_prediction_intervals(preds)

In [94]:
anoms = detector.detect(ts_test, preds)

In [95]:
plot_detection(ts_test, upper, lower, preds)

Loading BokehJS ...

In [96]:
a = anoms.index.tolist()
b = test.index.tolist()

for elem in a:
    if elem not in b:
        print(elem)

2022-01-09 03:50:00
2022-01-24 02:30:00
2022-01-24 12:10:00
2022-02-01 11:30:00
2022-02-01 11:40:00
2022-02-01 12:00:00
2022-02-04 22:30:00
2022-02-04 22:40:00
2022-02-08 08:50:00
2022-02-08 09:00:00
2022-02-22 04:00:00
2022-03-14 02:00:00
2022-03-16 13:10:00
